# Sweety Jewellers Cataloging Program v3

## Requirements

**Python 3.9**<br>
pandas: `pip install pandas`<br>
openpyxl: `conda install -c conda-forge openpyxl`<br>
fpdf: `pip install fpdf`<br>
tqdm: `pip install tqdm`<br>

### Testing
notebook: `conda install -c conda-forge notebook`

In [29]:
import os
import time
import logging
import pandas as pd

from datetime import datetime, timedelta
from PIL import Image, ImageFont, ImageDraw

In [30]:
if not os.path.exists(r"logs/"):
    logging.debug("No directory for logs found. Creating logs directory.")
    os.makedirs(r"logs/")

# Set up logging to file
LOGGING_DIR = r'logs/'
session_startTime = datetime.now()
local_tzname = (((session_startTime).astimezone()).tzinfo).tzname((session_startTime).astimezone())
logfile = session_startTime.strftime("Session_%d%b%y_%H%M%S.log")
# NOTE: File logging has been limited to INFO level
logging.basicConfig(format='%(name)s %(asctime)s: %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S', filename=LOGGING_DIR+logfile)

# Set up logging for display stdout
console = logging.StreamHandler()
# NOTE: File logging has been not restricted
console.setLevel(logging.NOTSET)
formatter = logging.Formatter('%(asctime)s: %(message)s')
console.setFormatter(formatter)
logging.getLogger().addHandler(console)

In [31]:
# SETTINGS
XLSX_ITEMS = "Items.xlsx"
XLSX_PURCHASES = "Catalogue Purchase.xlsx"
NEW_STOCK_DATE = datetime.utcnow()+timedelta(hours=5, minutes=30)-timedelta(days=28)
PRICE_COLS = ["D"]
USEFUL_COLS = ["Product Code", "Name", "Group", "Category", "Base Unit", "To Clear", "Min Qty", "Ord Unit"]
INP_FOLDER = "JPG"
FONT_SIZE = 45
FONT_FILE = r"assets/font.ttf"
LABELS_PER_LINE = 2
LINE_HEIGHT = 100
OUT_FOLDER = "Output"

In [32]:
# Read Main Excel File
df = pd.read_excel(XLSX_ITEMS, engine='openpyxl', skiprows=5, usecols=PRICE_COLS+USEFUL_COLS)

# Strip whitespace in str columns
for col in df.columns:
    if df.dtypes[col]=='O':
        df[col].str.strip()
        df.fillna(value={col: ""})

df.drop(index=0, inplace=True)

df.set_index("Product Code", inplace=True)

df["Min Qty"] = pd.to_numeric(df["Min Qty"], downcast="integer")

In [33]:
# Reading Purchase Order Excel File
purchase_data = pd.read_excel(XLSX_PURCHASES)

for col in df.columns:
    if df.dtypes[col]=='O':
        df[col].str.strip()
        df.fillna(value={col: ""})

purchase_data.set_index("Item Name", inplace=True)

In [34]:
df['time_tag'] = ""

In [35]:
for i in list(df.index):
    if df.at[i, "To Clear"] in ["Y", "y"]:
        df.at[i, 'time_tag'] = "To Clear"
        continue
    else:
        try:
            last_purchase_date = max(purchase_data.at[i, 'Date'])
        except KeyError as e:
            df.at[i, 'time_tag'] = ''
            continue
        except TypeError as e:
            last_purchase_date = purchase_data.at[i, 'Date']

        if last_purchase_date>=NEW_STOCK_DATE:
            df.at[i, 'time_tag'] = 'New This Month'
        else:
            df.at[i, 'time_tag'] = ''

In [36]:
for col_index in range(len(PRICE_COLS)):
    col = PRICE_COLS[col_index]
    new_col = "Rate_"+col

    df[new_col] = ""
    for i in list(df.index):
        if pd.isna(df.at[i, col]) is True:
            print(f"{i} is missing value in column {col}. This item will be skipped.")
        else:
            df.at[i, new_col] = f"Rs. {df.at[i, col]:.2f} per "+str(df.at[i, "Base Unit"])

    PRICE_COLS[col_index] = new_col

SJ-SSP-105 is missing value in column D. This item will be skipped.
SJ-SSP-22F33 is missing value in column D. This item will be skipped.


In [37]:
df["Min Ord"] = ""
for item in df.index:
    if pd.isna(df.at[item, "Min Qty"]) is True:
        print(f"{item} is missing value for Min Qty. This item will be skipped.")
    elif pd.isna(df.at[item, "Ord Unit"]) is True:
        print(f"{item} is missing value for Ord Unit. This item will be skipped.")
    else:
        df.at[item, "Min Ord"] = str(df.at[item, "Min Qty"]) + " " + df.at[item, "Ord Unit"]

In [39]:
df['ip_file_path'] = ''
filepaths = os.listdir(INP_FOLDER)
for item in df.index:
    if ((item+".jpg") in filepaths):
        df.at[item, 'ip_file_path'] = INP_FOLDER+r"/"+item+".jpg"
    elif ((item+".jpeg") in filepaths):
        df.at[item, 'ip_file_path'] = INP_FOLDER+r"/"+item+".jpeg"
    else:
        print(f"No image found for {item}.")
    

No image found for SJ-SSP-056.
No image found for SJ-SSP-132L.
No image found for SJ-SSP-21K11.
No image found for SJ-SSP-21K13.
No image found for SJ-SSP-22F33.
No image found for SJ-SSP-22F55.
No image found for SJ-SSP-22F57.


In [41]:
def create_images(df, primary_key, labels, rate_col=None):
    
    global FONT_SIZE, FONT_FILE, LINE_HEIGHT, LABELS_PER_LINE

    pipeline = list()
    total_height = 0
    
    raw_image = Image.open(df.at[primary_key, 'ip_file_path'])
    
    # Resizing image
    w,h = raw_image.size
    
    pipeline.append(raw_image)
    total_height += h

    font_size = int((w/2100)*FONT_SIZE)
    FONT_SETTINGS = ImageFont.truetype(FONT_FILE, font_size)
    LINE_HEIGHT = int((w/2100)*LINE_HEIGHT) # pixels
    
    
    # Blank line above
    im = Image.new(mode="RGB",size=(w, int(LINE_HEIGHT/2)), color=(250, 250, 250))
    pipeline.append(im)
    total_height += int(LINE_HEIGHT/2)
    
    # Preparing text to write
    txt_lines = [[]]
    for i in range(len(labels)):
        
        if len(txt_lines[-1])==LABELS_PER_LINE:
            txt_lines.append(list())
        
        if labels[i]==df.index.name:
            msg = labels[i]+": "+primary_key
        elif labels[i]=="Rate":
            msg = "Rate: "+str(df.at[primary_key, "Rate_"+col])
        else:
            msg = labels[i]+": "+str(df.at[primary_key, labels[i]])
        
        txt_lines[-1].append(msg)
    
    # Writing text to image line-by-line
    for lines in txt_lines:
        im = Image.new(mode="RGB",size=(w, LINE_HEIGHT), color=(250, 250, 250))
        draw = ImageDraw.Draw(im)
        msg = "        ".join(lines)
        draw.text((w/2, LINE_HEIGHT/2), msg, fill='black', font=FONT_SETTINGS, anchor='mm')
        pipeline.append(im)
        total_height += LINE_HEIGHT
    
    # Blank line below
    im = Image.new(mode="RGB",size=(w, int(LINE_HEIGHT/2)), color=(250, 250, 250))
    pipeline.append(im)
    total_height += int(LINE_HEIGHT/2)
    
    # Final Image
    final_img = Image.new(mode='RGB', size=(w, total_height))
    
    height_counter = 0
    for i in range(len(pipeline)):
        final_img.paste(pipeline[i], box=(0, height_counter))
        _, h = pipeline[i].size
        height_counter += h
    
    return final_img

In [42]:
if not os.path.isdir(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

#TODO: Split images into folders according to TIME_TAG ->  GROUP -> CATEGORY and GROUP -> CATEGORY -> TIME_TAG
for item in df.index:
    
    if df.at[item, "ip_file_path"]=='':
        continue
    
    if not os.path.isdir(OUT_FOLDER+r"/"+"Wholesale"):
        os.makedirs(OUT_FOLDER+r"/"+"Wholesale")
    img = create_images(df, item, labels=["Product Code", "Group", "Category"])
    img.save(OUT_FOLDER+r"/"+"Wholesale"+r"/"+item+".jpg")
    del(img)
    
    for price_col in PRICE_COLS:
        if not os.path.isdir(OUT_FOLDER+r"/"+price_col):
            os.makedirs(OUT_FOLDER+r"/"+price_col)
        if df.at[item, price_col]!='':
            img = create_images(df, item, labels=["Product Code", "Group", "Category", "Rate", "Min Ord"], rate_col=price_col)
            img.save(OUT_FOLDER+r"/"+price_col+r"/"+item+".jpg")
            del(img)

In [43]:
#TODO: Create PDF.